In [ ]:
# default_exp docstring

# Exporting Docstrings

> Converts `docment` docstrings to Numpy styled

In [ ]:
#hide
from nbdev.showdoc import *

The goal of this module is to take code that looks like the following:

In [ ]:
def addition(
    a:int, # The first number to add
    b:int=2, # The second number to add
) -> int: # The sum of a and b
    "Adds two numbers together"
    return a+b

And convert it to be the following:

In [ ]:
def addition(a,b) -> int:
    """Adds two numbers together
    
    Parameters
    ---------
    a : int
        The first number to add
    b : int
        The second number to add
        
    Returns
    -------
    int
        The sum of a and b
    """
    return a + b

In [ ]:
import nbdev.export as exp

In [ ]:
nb = exp.read_nb('99_test.ipynb')

In [ ]:
default = exp.find_default_export(nb['cells'])

In [ ]:
mod = exp.get_nbdev_module()

In [ ]:
exports = [exp.is_export(c, default) for c in nb['cells']]
cells = [(i,c,e) for i,(c,e) in enumerate(zip(nb['cells'],exports)) if e is not None]

In [ ]:
flag_lines, code_lines = exp.split_flags_and_code(cells[0][1])

In [ ]:
code_lines = exp._deal_import(code_lines, 'docstring.py')

In [ ]:
code_lines

['def addition(',
 '    a:int, # The first number to add',
 '    b:int, # The second number to add',
 ') -> int: # The sum of a and b',
 '    "Adds two numbers together"',
 '    return a+b']

In [ ]:
code = '\n'*(int(exp.get_config().get('cell_spacing', '1'))+1) + '\n' + '\n'.join(code_lines)

In [ ]:
code = '\n\n\ndef addition(\n    a:int, # The first number to add\n    b:int, # The second number to add\n) -> int: # The sum of a and b\n    "Adds two numbers together"\n    return a+b'

In [ ]:
import fastcore.docments as dments

In [ ]:
dments.docments(code)

{'a': 'The first number to add',
 'b': 'The second number to add',
 'return': 'The sum of a and b'}

In [ ]:
#export
from __future__ import annotations

In [ ]:
source = """def addition(
    a:(int, float), # The first number to add
    # The second number to add
    b:int = 2,
) -> (int,float): # The sum of a and b
    "Adds two numbers together"
    return a+b
"""

In [ ]:
def addition(
    a:(int, float), # The first number to add
    # The second number to add
    b:int = 2,
) -> (int, float): # The sum of a and b
    "Adds two numbers together"
    return a+b

In [ ]:
dments.docments(addition)

{'a': 'The first number to add',
 'b': 'The second number to add',
 'return': 'The sum of a and b'}

In [ ]:
#export
def get_annotations(
    source:str # Source code of function or class
):
    "Extracts the type annotations from source code"
    annos = []
    orig = list(locals().keys())
    exec(source);
    new = list(locals().keys())
    orig += ['new', 'orig']
    new += ['new']
    for key in set(new) - set(orig):
        params = inspect.signature(locals()[key]).parameters.keys()
        param_args = []
        for param in params:
            param_args.append(inspect.signature(locals()[key]).parameters[param].default)
        return_anno = inspect.signature(locals()[key]).return_annotation
        annos.append((
            locals()[key].__annotations__, 
            locals()[key].__doc__,
            param_args,
            return_anno
        ))
        del locals()[key]
    if len(annos) == 1: return annos[0]
    else: return annos

In [ ]:
#export
def reformat_function(
    source:str, # Source code
):
    "Takes messy source code and refactors it into a readable PEP-8 standard style"
    docs = dments.docments(source)
    annos = get_annotations(source)
    param_locs = dments._param_locs(source)
    new_source = ''
    new_source += source.split('\n')[0]
    for i, (name, default) in enumerate(zip(annos[0].keys(), annos[2])):
        new_source += f'{name}'
        if default != inspect._empty:
            new_source += f'={default}'
        if i < len(annos[2])-1:
            new_source += ','
        else:
            new_source += '):'
    new_source.rstrip(',')
    _ds = '    '
    new_source += f'\n{_ds}"""\n{_ds}'
    new_source += f'{annos[1]}\n\n'
    new_source += f'{_ds}Parameters\n{_ds}----------\n'
    for param in param_locs.values():
        if param != 'return':
            new_source += f'{_ds}{param} : {annos[0][param]}\n\t'
            new_source += docs[param] + '\n'
    if annos[-1] != inspect._empty:
        new_source += f'\n{_ds}Returns:\n{_ds}--------\n'
        new_source += f'{_ds}{annos[-1]}\n\t{docs["return"]}\n'
    new_source += f'{_ds}"""'
    new_source += source.split('"')[-1]
    new_source = new_source.rstrip('\n')
    return new_source

In [ ]:
class Adder:
    a = 2
    b = 5
    def __init__(
        self,
        a:int, # A starting value
        b:(int,float), # A number to add by
    ):
        "Test"
        self.a = a
        self.b = b
        
    def add(self):
        "Adds self.a and self.b"
        return self.a + self.b

In [ ]:
source = """class Adder:
    def __init__(
        self,
        a:int, # A starting value
        b:(int,float), # A number to add by
    ):
        "Test"
        self.a = a
        self.b = b
        
    def add(self):
        "Adds self.a and self.b"
        return self.a + self.b"""

In [ ]:
import ast
dir(ast.parse(source).body[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'bases',
 'body',
 'col_offset',
 'decorator_list',
 'keywords',
 'lineno',
 'name']

In [ ]:
ast.parse(source).body[0]

[]

In [ ]:
ast.parse(source).body[0].body[0].name

'__init__'

In [ ]:
import asttokens

In [ ]:
atok = asttokens.ASTTokens(source, parse=True)

In [ ]:
num_funcs = sum(isinstance(exp, ast.FunctionDef) for exp in nodes)

In [ ]:
sources = []
func_line_nos = []
curr_func_source = []
curr_func = 0
line_nums = []
for i, node in enumerate(ast.walk(atok.tree)):
    if isinstance(node, ast.FunctionDef):
        func_line_nos.append(node.lineno)
    else:
        if len(func_line_nos) > 0 and hasattr(node, 'lineno'):
            if node.lineno > func_line_nos[curr_func]: # If the current line number is larger than the definition line number
#                 print(node.lineno, func_line_nos[curr_func])
                # Check if we need to move to the next function
                if curr_func + 1 < len(func_line_nos):
                    if node.lineno < func_line_nos[curr_func+1]:
                        # If we're between the next function definition
                        if node.lineno not in line_nums:
#                             print("B")
                            curr_func_source.append(atok.get_text(node))
                            line_nums.append(node.lineno)
                    else:
#                         print("C")
                        sources.append(curr_func_source)
                        curr_func_source = []
                        curr_func += 1
                elif node.lineno > func_line_nos[curr_func]:
                    if node.lineno not in line_nums:
                        curr_func_source.append(atok.get_text(node))
                        line_nums.append(node.lineno)
                
            elif curr_func == len(func_line_nos) - 1:
                if node.lineno not in line_nums:
                    curr_func_source.append(atok.get_text(node))
                    line_nums.append(node.lineno)

Line: class Adder:
    def __init__(
        self,
        a:int, # A starting value
        b:(int,float), # A number to add by
    ):
        "Test"
        self.a = a
        self.b = b
        
    def add(self):
        "Adds self.a and self.b"
        return self.a + self.b
Line: class Adder:
    def __init__(
        self,
        a:int, # A starting value
        b:(int,float), # A number to add by
    ):
        "Test"
        self.a = a
        self.b = b
        
    def add(self):
        "Adds self.a and self.b"
        return self.a + self.b
Line: self,
        a:int, # A starting value
        b:(int,float)
Line: self
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 
Line: 


In [ ]:
sources

[['"Test"', 'self.a = a', 'self.b = b']]

In [ ]:
func_line_nos

[2, 11]

In [ ]:
curr_func_source

['return self.a + self.b',
 'self',
 'a:int',
 'b:(int,float)',
 'self',
 '"Adds self.a and self.b"']

In [ ]:
line_nums

[]

In [ ]:
curr_func

1

In [ ]:
source = """class Adder:
    def __init__(
        self,
        a:int, # A starting value
        b:(int,float), # A number to add by
    ):
        "Test"
        self.a = a
        self.b = b
        
    def add(self):
        "Adds self.a and self.b"
        return self.a + self.b"""

['"Adds self.a and self.b"',
 'return self.a + self.b',
 '"Adds self.a and self.b"',
 'self.a + self.b',
 'self.a',
 'self.b',
 'self',
 'self']

In [ ]:
curr_func_source

[]

In [ ]:
nodes[1].value.lineno

8

In [ ]:
dir(nodes[-1])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'col_offset',
 'ctx',
 'first_token',
 'id',
 'last_token',
 'lineno']

In [ ]:
help(dis.dis)

Help on function dis in module dis:

dis(x=None, *, file=None, depth=None)
    Disassemble classes, methods, functions, and other compiled objects.
    
    With no argument, disassemble the last traceback.
    
    Compiled objects currently include generator objects, async generator
    objects, and coroutine objects, all of which store their code object
    in a special attribute.



In [ ]:
dis.dis(x=source)

  1           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               0 (<code object Adder at 0x7ff1bba209c0, file "<dis>", line 1>)
              4 LOAD_CONST               1 ('Adder')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               1 ('Adder')
             10 CALL_FUNCTION            2
             12 STORE_NAME               0 (Adder)
             14 LOAD_CONST               2 (None)
             16 RETURN_VALUE

Disassembly of <code object Adder at 0x7ff1bba209c0, file "<dis>", line 1>:
  1           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('Adder')
              6 STORE_NAME               2 (__qualname__)

  4           8 LOAD_NAME                3 (int)

  5          10 LOAD_NAME                3 (int)
             12 LOAD_NAME                4 (float)
             14 BUILD_TUPLE              2
             16 LOAD_CONST               1 (('a'

In [ ]:
decoded

In [ ]:
funcs = ast.parse(source).body[0].body
for func in funcs:
    if isinstance(func, ast.FunctionDef):
        reformat_function()

In [ ]:
ast.parse(source).body[0].body[1].name

'add'

In [ ]:
ast.parse(source).body[0].body[1].col_offset # Use this to get the number of spaces in an offset

4

In [ ]:
#export
def reformat_function(
    source:str, # Source code
):
    "Takes messy source code and refactors it into a readable PEP-8 standard style"
    docs = dments.docments(source)
    annos = get_annotations(source)
    param_locs = dments._param_locs(source)
    new_source = ''
    new_source += source.split('\n')[0]
    for i, (name, default) in enumerate(zip(annos[0].keys(), annos[2])):
        new_source += f'{name}'
        if default != inspect._empty:
            new_source += f'={default}'
        if i < len(annos[2])-1:
            new_source += ','
        else:
            new_source += '):'
    new_source.rstrip(',')
    _ds = '    '
    new_source += f'\n{_ds}"""\n{_ds}'
    new_source += f'{annos[1]}\n\n'
    new_source += f'{_ds}Parameters\n{_ds}----------\n'
    for param in param_locs.values():
        if param != 'return':
            new_source += f'{_ds}{param} : {annos[0][param]}\n\t'
            new_source += docs[param] + '\n'
    if annos[-1] != inspect._empty:
        new_source += f'\n{_ds}Returns:\n{_ds}--------\n'
        new_source += f'{_ds}{annos[-1]}\n\t{docs["return"]}\n'
    new_source += f'{_ds}"""'
    new_source += source.split('"')[-1]
    new_source = new_source.rstrip('\n')
    return new_source

In [ ]:
dments.docments(Adder)

{'self': None, 'a': 'A starting value', 'b': 'A number to add by'}

In [ ]:
class Subtracter(Adder):
    def subtract(self):
        "Subtracts self.a and self.b"
        return self.a - self.b

For classes we'll need to extract functions and constants, and only reformat the functions in each class

In [ ]:
class ExampleClass(object):
    """The summary line for a class docstring should fit on one line.

    If the class has public attributes, they may be documented here
    in an ``Attributes`` section and follow the same formatting as a
    function's ``Args`` section. Alternatively, attributes may be documented
    inline with the attribute's declaration (see __init__ method below).

    Properties created with the ``@property`` decorator should be documented
    in the property's getter method.

    Attributes
    ----------
    attr1 : str
        Description of `attr1`.
    attr2 : :obj:`int`, optional
        Description of `attr2`.

    """

    def __init__(self, param1, param2, param3):
        """Example of docstring on the __init__ method.

        The __init__ method may be documented in either the class level
        docstring, or as a docstring on the __init__ method itself.

        Either form is acceptable, but the two should not be mixed. Choose one
        convention to document the __init__ method and be consistent with it.

        Note
        ----
        Do not include the `self` parameter in the ``Parameters`` section.

        Parameters
        ----------
        param1 : str
            Description of `param1`.
        param2 : :obj:`list` of :obj:`str`
            Description of `param2`. Multiple
            lines are supported.
        param3 : :obj:`int`, optional
            Description of `param3`.

        """
        self.attr1 = param1
        self.attr2 = param2
        self.attr3 = param3  #: Doc comment *inline* with attribute

        #: list of str: Doc comment *before* attribute, with type specified
        self.attr4 = ["attr4"]

        self.attr5 = None
        """str: Docstring *after* attribute, with type specified."""

    @property
    def readonly_property(self):
        """str: Properties should be documented in their getter method."""
        return "readonly_property"

    @property
    def readwrite_property(self):
        """:obj:`list` of :obj:`str`: Properties with both a getter and setter
        should only be documented in their getter method.

        If the setter method contains notable behavior, it should be
        mentioned here.
        """
        return ["readwrite_property"]

    @readwrite_property.setter
    def readwrite_property(self, value):
        value

    def example_method(self, param1, param2):
        """Class methods are similar to regular functions.

        Note
        ----
        Do not include the `self` parameter in the ``Parameters`` section.

        Parameters
        ----------
        param1
            The first parameter.
        param2
            The second parameter.

        Returns
        -------
        bool
            True if successful, False otherwise.

        """
        return True

    def __special__(self):
        """By default special members with docstrings are not included.

        Special members are any methods or attributes that start with and
        end with a double underscore. Any special member with a docstring
        will be included in the output, if
        ``napoleon_include_special_with_doc`` is set to True.

        This behavior can be enabled by changing the following setting in
        Sphinx's conf.py::

            napoleon_include_special_with_doc = True

        """
        pass

    def __special_without_docstring__(self):
        pass

    def _private(self):
        """By default private members are not included.

        Private members are any methods or attributes that start with an
        underscore and are *not* special. By default they are not included
        in the output.

        This behavior can be changed such that private members *are* included
        by changing the following setting in Sphinx's conf.py::

            napoleon_include_private_with_doc = True

        """
        pass

    def _private_without_docstring(self):
        pass